<a href="https://colab.research.google.com/github/dssp-ethz/DSSP-Illustrations/blob/main/DSSP_From_Discrete_Time_to_Continuous_Time_and_Back.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sampling Theorem

## Problem 2.6 (as of 2020)

Let $F(i2\pi f) = \begin{cases} 1 - |f|/f_0 & |f| < f_0\\ 0 & \text{otherwise}\end{cases}$

Sketch the spectrum $F_s(e^{i \Omega})$ of the sampled signal $f_s [k] = T f(kT)$ for 

1.   $T = 1/(2f_0)$
2.   $T = 1/(\frac{3}{2}f_0)$
3.   $T = 1/(f_0)$

$Y_s(z) = \sum\limits_{s\in\mathbb{C} : e^{sT} = z} Y(s) \qquad$ (2.48 in lecture notes)


